In [43]:
######################## Cleaning The Tweets ##########################
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
filename = 'train.csv'
df = pd.read_csv(filename)
stop = stopwords.words('english')
lemmatizer = WordNetLemmatizer()

df['tweet'] = df['tweet'].apply(lambda x: p.clean(re.sub('#', '',x))) #Tweet Cleaner
df['tweet'] = df['tweet'].apply(lambda x: x.lower()) #Lower Casing
df['tweet'] = df['tweet'].str.replace('[^\w\s]','') #Removing Punctuations
df['tweet'] = df['tweet'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))#STopwords Removal
df['tweet'] = df['tweet'].apply(lambda x: " ".join([lemmatizer.lemmatize(word) for word in x.split()]))

df.to_csv('clean_'+str(filename))

In [1]:
import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt
import string

from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import TweetTokenizer

import preprocessor as p
from tqdm import tqdm

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [2]:
df = pd.read_csv('clean_train.csv')
df.head()

,id,label,tweet
0,1,0,father dysfunctional selfish drag kid dysfunct...
1,2,0,thanks lyft credit cant use cause dont offer w...
2,3,0,bihday majesty
3,4,0,model love u take u time ur
4,5,0,factsguide society motivation


In [3]:
df = df.dropna()
normal = df[df['label']==0]
hate = df[df['label']==1]

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer  
tfidfconverter = TfidfVectorizer(max_features=10000,ngram_range=(1,2), min_df=5, max_df=0.7, stop_words=stopwords.words('english'))  
X = tfidfconverter.fit_transform(df['tweet']).toarray()

In [12]:
from sklearn.model_selection import train_test_split  
X_train, X_test, y_train, y_test = train_test_split(X, df['label'], test_size=0.75, random_state=0)

In [10]:
from sklearn.naive_bayes import GaussianNB, MultinomialNB

gnb = GaussianNB()
gnb.fit(X_train, y_train)
predictions = gnb.predict(X_test)
#################### Gaussian NB Result #########################
print(confusion_matrix(y_test,predictions))  
print(classification_report(y_test,predictions))  
print(accuracy_score(y_test, predictions))

[[6526  880]
 [ 221  356]]
              precision    recall  f1-score   support

           0       0.97      0.88      0.92      7406
           1       0.29      0.62      0.39       577

    accuracy                           0.86      7983
   macro avg       0.63      0.75      0.66      7983
weighted avg       0.92      0.86      0.88      7983

0.8620819240886884


In [14]:
from sklearn.ensemble import RandomForestClassifier
text_classifier = RandomForestClassifier(n_estimators=30, random_state=0)  
text_classifier.fit(X_train, y_train)
predictions = text_classifier.predict(X_test)

print(confusion_matrix(y_test,predictions))  
print(classification_report(y_test,predictions))  
print(accuracy_score(y_test, predictions))

[[22191    96]
 [ 1003   659]]
              precision    recall  f1-score   support

           0       0.96      1.00      0.98     22287
           1       0.87      0.40      0.55      1662

    accuracy                           0.95     23949
   macro avg       0.91      0.70      0.76     23949
weighted avg       0.95      0.95      0.95     23949

0.9541108188233329
